In [ ]:
from pathlib import Path
import json
import pandas as pd
from uau_api.settings import Settings
from IPython.display import display, HTML
from requests.compat import urlparse, unquote
from uau_api import UauAPI
from uau_api.utils import write_jsonl
import hashlib
import base64
from PIL import Image
from io import BytesIO

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
uau = UauAPI(Settings().API_URL, Settings().API_KEY)
uau.authenticate('leonardo', 'hybr01')
wd = Path('/mnt/f')
xl_filename = wd / 'NFs.xlsx'
xl = pd.ExcelFile(xl_filename)
df = xl.parse(xl.sheet_names[0])

In [ ]:
empresas = uau.Empresa.obter_empresas_ativas()
empresas = pd.json_normalize(empresas)
obras = uau.Obras.obter_obras_ativas()
obras = pd.json_normalize(obras)
empresas = empresas.merge(obras, how='inner', left_on='Codigo_emp', right_on='Empresa_obr')
# df = df.merge(empresas[['Codigo_emp', 'Desc_emp', 'Cod_obr']], how='left', left_on='Descrição empresa escrituração', right_on='Desc_emp')

In [ ]:
s1 = empresas.query('Desc_emp.str.contains("GMW")', engine='python')['Codigo_emp']

In [ ]:
df['Dt. cadastro']

In [ ]:
s2 = df[['CNPJ/CPF', 'Dt. cadastro', 'Dt. emissão']]

In [ ]:
df2 = pd.merge(left=s1, right=s2, how='cross')

In [ ]:
df2= df2.drop_duplicates()

In [ ]:
def hash_dict(d):
    # Convert items to sorted tuples for consistent hashing
    sorted_items = tuple(sorted(d.items()))
    # Use a cryptographic hash for robustness, or hash() for simple cases
    return hashlib.sha256(str(sorted_items).encode()).hexdigest()


def save_base64_str(base64_str, filename):
    if base64_str.startswith('data:'):
        base64_str = base64_str.split(',')[1]

    with open(filename, 'wb') as f:
        f.write(base64.b64decode(base64_str))


def base64_to_pillow_image(string):
    return Image.open(BytesIO(base64.b64decode(string)))


def pillow_image_to_pdf(image: Image, file_path):
    image.save(file_path, 'PDF', resolution=100.0)

In [ ]:
found: set = set()
notas = []
for _, row in df2.iterrows():
    print('processed: %d' % _)

    dict_hash = hash_dict(row.to_dict())
    if dict_hash in found:
        print('skip', flush=True)
        continue
    response = uau.NotasFiscais.consultar_nfentrada(
        codigo_empresa=int(row['Codigo_emp']),
        cnpj_fornecedor=str(row['CNPJ/CPF']),
        data_inicial=min([row['Dt. cadastro'], row['Dt. emissão']]).isoformat(),
        data_final=max([row['Dt. cadastro'], row['Dt. emissão']]).isoformat(),
        tipo_periodo=1,
    )
    if response:
        found.add(dict_hash)
        notas.extend(response)
write_jsonl(notas, 'notas.jsonl')

In [ ]:
df_notas = pd.json_normalize(notas)
#, record_path='Processos', meta=['Empresa', 'Numero', 'CodigoFornecedor', 'CNPJCPFFornecedor'], record_prefix='Processos.')
df_notas = df_notas[['Empresa', 'Numero', 'CodigoFornecedor', 'CNPJCPFFornecedor', 'Processos']].dropna(subset='Empresa')
df_notas = df_notas.explode(column='Processos')
df_notas = pd.concat([df_notas.drop(['Processos'], axis=1), df_notas['Processos'].apply(pd.Series).add_prefix('Processos.')], axis=1)
df_notas.iloc[0]

In [ ]:
files_found: set = set()

In [ ]:
for _, row in df_notas.dropna(subset='Processos.Numero').iterrows():
    empresa = int(row['Empresa'])
    obra = str(row['Processos.Obra'])
    processo = int(row['Processos.Numero'])
    numero_nota = str(row['Numero'])
    notas_dir = wd / f'nf/{empresa}/{obra}'
    notas_dir.mkdir(parents=True, exist_ok=True)
    arquivos = uau.Anexo.retornar_arquivos_em_lista_bytes(
        empresa=int(empresa), identificador=f'PROCESSO {int(processo)}-{str(obra)}'
    )
    if arquivos:
        for index, arquivo in enumerate(arquivos):
            if isinstance(arquivo, dict):
                nome_arquivo = Path(arquivo['NomeArquivo'])
                file_name = f'{empresa}-{obra}-{processo}-{numero_nota}-arquivo[{index}][{nome_arquivo.stem}]{nome_arquivo.suffix.lower()}'
                pdf_name = f'{empresa}-{obra}-{processo}-{numero_nota}-arquivo[{index}][{nome_arquivo.stem}].pdf'
                file_path = notas_dir / file_name
                pdf_path = notas_dir / pdf_name
                arquivo['index'] = index
                write_jsonl(
                    row.to_dict() | arquivo | {'imagem': str(file_path.resolve()), 'pdf': str(pdf_path.resolve())},
                    wd / 'arquivos_notas.jsonl',
                    mode='a',
                )
                dict_hash = hash_dict(arquivo)
                if dict_hash in files_found:
                    print(f'Duplicated {file_name}')
                    continue
                save_base64_str(arquivo['ConteudoArquivo'], file_path)
                image = base64_to_pillow_image(arquivo['ConteudoArquivo'])
                pillow_image_to_pdf(image, pdf_path)
                files_found.add(dict_hash)
                print(file_name)

In [264]:
with open(wd / 'arquivos_notas.jsonl') as f:
    file =[json.loads(line) for line in f.readlines()]

In [270]:
pd.json_normalize(file).to_excel(wd / 'notas_fiscais.xlsx', index=False, float_format='%.2f')